# Projeto Solar — Execução Parametrizada

- Ajuste a célula de parâmetros e execute.
- Não é necessário usar `input()` nesta versão.
- Certifique-se de executar a célula de "Setup do notebook" antes das demais.

In [ ]:
# Setup do notebook: garantir acesso ao pacote 'src' e autoreload
import os, sys
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

# Recarregar módulos automaticamente ao salvar
try:
    %reload_ext autoreload
    %autoreload 2
except Exception:
    pass

In [ ]:
# Parâmetros de entrada (ajuste e execute esta célula)
CIDADE = "Fortaleza, CE"
CONSUMO_KWH = 350.0         # consumo médio mensal em kWh
TIPO_CONEXAO = 50           # 30 (mono), 50 (bi) ou 100 (tri)
FINANCIAR = False           # True/False
TAXA_AA = 20.0              # taxa anual (% a.a.) se financiar
MESES = 60                  # prazo em meses se financiar

# Execução não interativa
irr, temp = get_data(CIDADE)
if irr is None:
    raise RuntimeError("Cidade não encontrada. Ajuste 'CIDADE'.")

fin_data = (TAXA_AA, MESES) if FINANCIAR else None
qtd, pot, capex, parc, ant, novo, saldo, tot_sem, tot_com = calcular_tudo(
    CONSUMO_KWH, TIPO_CONEXAO, irr, temp, FINANCIAR, fin_data
)

# Resumo rápido
print("\n💰 RESUMO COMERCIAL:")
print(f"   Investimento Total: R$ {capex:,.2f}")
if FINANCIAR:
    print(f"   Parcela Mensal: R$ {parc:,.2f}")

# Gráficos e imagem
plotar_dashboard_final(CIDADE, pot, ant, novo, saldo, tot_sem, tot_com, parc, FINANCIAR)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.geodata import get_data
from src.engineering import calcular_tudo, imprimir_relatorio_tecnico
from src.finance import calcular_financiamento
from src.viz import plotar_dashboard_final

# ==============================================================================
# 🗄️ DADOS TÉCNICOS
# ==============================================================================
DB_HARDWARE = {
    'MODULO': {'W': 555, 'AREA': 2.6, 'PESO': 28}, # 555W / 2.6m² / 28kg
}

# ==============================================================================
# 🛠️ FUNÇÃO PRINCIPAL
# ==============================================================================
def main():
    print("\n=== ☀️ SOLAR SYSTEM V11 (HÍBRIDO) ☀️ ===")
    while True:
        c = input("\nCidade: ").strip()
        if c.lower() == 'sair': return
        irr, temp = get_data(c)
        if irr is not None: break
        print("❌ Cidade não encontrada. Tente 'Macapa, AP'")

    cons = float(input("Média de Consumo (kWh): ").strip().replace(',', '.'))
    taxa = int(input("Escolha o tipo de conexão (30kWh, 50kWh, 100kWh): ").strip())

    fin = input("Cliente quer financiar? (s/n): ").strip().lower() == 's'
    fin_data = (float(input("Taxa (% a.a): ").strip()), int(input("Meses: ").strip())) if fin else None

    print("\n⏳ Calculando Engenharia e Financeiro...")
    qtd, pot, capex, parc, ant, novo, saldo, tot_sem, tot_com = calcular_tudo(cons, taxa, irr, temp, fin, fin_data)

    print(f"\n💰 RESUMO COMERCIAL:")
    print(f"   Investimento Total: R$ {capex:,.2f}")
    if fin: print(f"   Parcela Mensal: R$ {parc:,.2f}")

    input("\n[ENTER] para abrir os Gráficos...")
    plotar_dashboard_final(c, pot, ant, novo, saldo, tot_sem, tot_com, parc, fin)

# ==============================================================================
# EXECUÇÃO
# ==============================================================================
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'pandas'